Taproot PSBT example from rust-bitcoin
https://github.com/rust-bitcoin/rust-bitcoin/blob/master/bitcoin/examples/taproot-psbt.rs

In [2]:
// :dep bitcoin = { path = "../../rust-bitcoin/bitcoin", features = ["std", "rand-std", "bitcoinconsensus"] }
:dep bitcoin = { version = "0.31.0-rc2", features = ["std", "rand-std", "bitcoinconsensus"] }
:dep secp256k1 = { version = "0.28.0", default-features = false, features = ["hashes"] }

In [3]:
use std::collections::BTreeMap;
use std::str::FromStr;

use bitcoin::consensus::encode;
use bitcoin::hashes::Hash;
use bitcoin::key::{TapTweak, XOnlyPublicKey};
use bitcoin::opcodes::all::{OP_CHECKSIG, OP_CLTV, OP_DROP};
use bitcoin::psbt::{self, Input, Output, Psbt, PsbtSighashType};
use bitcoin::secp256k1::Secp256k1;
use bitcoin::sighash::{self, SighashCache, TapSighash, TapSighashType};
use bitcoin::taproot::{self, LeafVersion, TapLeafHash, TaprootBuilder, TaprootSpendInfo};
use bitcoin::bip32::{ChildNumber, DerivationPath, Fingerprint, Xpriv, Xpub};
use bitcoin::{
     absolute, script, transaction, Address, Amount, Network, OutPoint, ScriptBuf, Transaction,
     TxIn, TxOut, Witness,
};

In [4]:
const BENEFACTOR_XPRIV_STR: &str = "tprv8ZgxMBicQKsPeRA8qujKduQRTYn5WTXYWEHEa1ifz7EVj9FofrU4mZBEMfJAZoUmCE3xWe2Ms8enPtBhHgMevwvHajRTbhB6EQr3RAGa5xw";
const BENEFICIARY_XPRIV_STR: &str = "tprv8ZgxMBicQKsPeesij9mBHF9Ca61CyKdCnuhgWwvKQKsAYjfLjfHuVUUz4HhgZqgDESnT5NujZ7WayQMYttre7NuupmeeJN4fbUMykxep4w6";
const BIP86_DERIVATION_PATH: &str = "m/86'/1'/0'/0/0";

In [5]:

const UTXO_SCRIPT_PUBKEY: &str =
    "51207468dfc1e2ff390804a4dfa4aa34d81302a7b3d1dd764cf882a30cda72a9ae30";
const UTXO_PUBKEY: &str = "d32e69a4c0096a6cb8569e8f27a0753b7d57956e6a69898111187e3d61cdee08";
const UTXO_MASTER_FINGERPRINT: &str = "4e6f8ac3";
const ABSOLUTE_FEES_IN_SATS: Amount = Amount::from_sat(1_000);

In [6]:

struct P2trUtxo<'a> {
    txid: &'a str,
    vout: u32,
    script_pubkey: &'a str,
    pubkey: &'a str,
    master_fingerprint: &'a str,
    amount_in_sats: Amount,
    derivation_path: &'a str,
}


In [7]:
// UTXO_1 will be used for spending example 1
const UTXO_1: P2trUtxo = P2trUtxo {
    txid: "1f44b9c4a5d06a2146ca0501bd7e2b4ea14ff7e5cc4b6296ebac5a07e6592d12",
    vout: 0,
    script_pubkey: UTXO_SCRIPT_PUBKEY,
    pubkey: UTXO_PUBKEY,
    master_fingerprint: UTXO_MASTER_FINGERPRINT,
    amount_in_sats: Amount::from_int_btc(25),
    derivation_path: BIP86_DERIVATION_PATH,
};


// UTXO_2 will be used for spending example 2
const UTXO_2: P2trUtxo = P2trUtxo {
    txid: "4a0dadb75e5ea88ee21371904f9d1bc0406612f76e303d87429bf793707b40d8",
    vout: 0,
    script_pubkey: UTXO_SCRIPT_PUBKEY,
    pubkey: UTXO_PUBKEY,
    master_fingerprint: UTXO_MASTER_FINGERPRINT,
    amount_in_sats: Amount::from_int_btc(25),
    derivation_path: BIP86_DERIVATION_PATH,
};

// UTXO_3 will be used for spending example 3
const UTXO_3: P2trUtxo = P2trUtxo {
    txid: "55025f328a6e796879a1dcaa88a62ba0b1a04d069c0774a077a5e8b13ff92aaf",
    vout: 1,
    script_pubkey: UTXO_SCRIPT_PUBKEY,
    pubkey: UTXO_PUBKEY,
    master_fingerprint: UTXO_MASTER_FINGERPRINT,
    amount_in_sats: Amount::from_int_btc(25),
    derivation_path: BIP86_DERIVATION_PATH,
};

In [11]:

// Lifted and modified from BDK at https://github.com/bitcoindevkit/bdk/blob/8fbe40a9181cc9e22cabfc04d57dac5d459da87d/src/wallet/signer.rs#L469-L503

// Bitcoin Dev Kit
// Written in 2020 by Alekos Filini <alekos.filini@gmail.com>
//
// Copyright (c) 2020-2021 Bitcoin Dev Kit Developers
//
// This file is licensed under the Apache License, Version 2.0 <LICENSE-APACHE
// or http://www.apache.org/licenses/LICENSE-2.0> or the MIT license
// <LICENSE-MIT or http://opensource.org/licenses/MIT>, at your option.
// You may not use this file except in accordance with one or both of these
// licenses.

// Calling this with `leaf_hash` = `None` will sign for key-spend
fn sign_psbt_taproot(
    secret_key: &secp256k1::SecretKey,
    pubkey: XOnlyPublicKey,
    leaf_hash: Option<TapLeafHash>,
    psbt_input: &mut psbt::Input,
    hash: TapSighash,
    hash_ty: TapSighashType,
    // secp: &Secp256k1<All>,
    secp: &Secp256k1<secp256k1::All>,
) {
    let keypair = secp256k1::Keypair::from_seckey_slice(secp, secret_key.as_ref()).unwrap();
    let keypair = match leaf_hash {
        None => keypair.tap_tweak(secp, psbt_input.tap_merkle_root).to_inner(),
        Some(_) => keypair, // no tweak for script spend
    };

    // TODO: After upgrade of secp change this to Message::from_digest(sighash.to_byte_array()).
    let msg =
        secp256k1::Message::from_digest(hash.to_byte_array());
        // secp256k1::Message::from_slice(hash.as_byte_array()).expect("tap sighash is 32 bytes long");
    let sig = secp.sign_schnorr(&msg, &keypair);

    let final_signature = taproot::Signature { sig, hash_ty };

    if let Some(lh) = leaf_hash {
        psbt_input.tap_script_sigs.insert((pubkey, lh), final_signature);
    } else {
        psbt_input.tap_key_sig = Some(final_signature);
    }
}


In [12]:
#[allow(clippy::single_element_loop)]
fn generate_bip86_key_spend_tx(
    secp: &secp256k1::Secp256k1<secp256k1::All>,
    master_xpriv: Xpriv,
    input_utxo: P2trUtxo,
    outputs: Vec<TxOut>,
) -> Result<Transaction, Box<dyn std::error::Error>> {
    let from_amount = input_utxo.amount_in_sats;
    let input_pubkey = XOnlyPublicKey::from_str(input_utxo.pubkey)?;

    // CREATOR + UPDATER
    let tx1 = Transaction {
        version: transaction::Version::TWO,
        lock_time: absolute::LockTime::ZERO,
        input: vec![TxIn {
            previous_output: OutPoint { txid: input_utxo.txid.parse()?, vout: input_utxo.vout },
            script_sig: ScriptBuf::new(),
            sequence: bitcoin::Sequence(0xFFFFFFFF), // Ignore nSequence.
            witness: Witness::default(),
        }],
        output: outputs,
    };
    let mut psbt = Psbt::from_unsigned_tx(tx1)?;

    let mut origins = BTreeMap::new();
    origins.insert(
        input_pubkey,
        (
            vec![],
            (
                Fingerprint::from_str(input_utxo.master_fingerprint)?,
                DerivationPath::from_str(input_utxo.derivation_path)?,
            ),
        ),
    );

    let mut input = Input {
        witness_utxo: {
            let script_pubkey = ScriptBuf::from_hex(input_utxo.script_pubkey)
                .expect("failed to parse input utxo scriptPubkey");
            Some(TxOut { value: from_amount, script_pubkey })
        },
        tap_key_origins: origins,
        ..Default::default()
    };
    let ty = PsbtSighashType::from_str("SIGHASH_ALL")?;
    input.sighash_type = Some(ty);
    input.tap_internal_key = Some(input_pubkey);
    psbt.inputs = vec![input];

    // The `Prevouts::All` array is used to create the sighash to sign for each input in the
    // `psbt.inputs` array, as such it must be the same length and in the same order as the inputs.
    let mut input_txouts = Vec::<TxOut>::new();
    for input in [&input_utxo].iter() {
        input_txouts.push(TxOut {
            value: input.amount_in_sats,
            script_pubkey: ScriptBuf::from_hex(input.script_pubkey)?,
        });
    }

    // SIGNER
    let unsigned_tx = psbt.unsigned_tx.clone();
    psbt.inputs.iter_mut().enumerate().try_for_each::<_, Result<(), Box<dyn std::error::Error>>>(
        |(vout, input)| {
            let hash_ty = input
                .sighash_type
                .and_then(|psbt_sighash_type| psbt_sighash_type.taproot_hash_ty().ok())
                .unwrap_or(TapSighashType::All);
            let hash = SighashCache::new(&unsigned_tx).taproot_key_spend_signature_hash(
                vout,
                &sighash::Prevouts::All(input_txouts.as_slice()),
                hash_ty,
            )?;

            let (_, (_, derivation_path)) = input
                .tap_key_origins
                .get(&input.tap_internal_key.ok_or("Internal key missing in PSBT")?)
                .ok_or("Missing taproot key origin")?;

            let secret_key = master_xpriv.derive_priv(secp, &derivation_path)?.to_priv().inner;
            sign_psbt_taproot(
                &secret_key,
                input.tap_internal_key.unwrap(),
                None,
                input,
                hash,
                hash_ty,
                secp,
            );

            Ok(())
        },
    )?;

    // FINALIZER
    psbt.inputs.iter_mut().for_each(|input| {
        let mut script_witness: Witness = Witness::new();
        script_witness.push(input.tap_key_sig.unwrap().to_vec());
        input.final_script_witness = Some(script_witness);

        // Clear all the data fields as per the spec.
        input.partial_sigs = BTreeMap::new();
        input.sighash_type = None;
        input.redeem_script = None;
        input.witness_script = None;
        input.bip32_derivation = BTreeMap::new();
    });

    // EXTRACTOR
    let tx = psbt.extract_tx_unchecked_fee_rate();
    tx.verify(|_| {
        Some(TxOut {
            value: from_amount,
            script_pubkey: ScriptBuf::from_hex(input_utxo.script_pubkey).unwrap(),
        })
    })
    .expect("failed to verify transaction");

    Ok(tx)
}


In [13]:
let secp = Secp256k1::new();

    println!("\n----------------");
    println!("\nSTART EXAMPLE 1 - P2TR with a BIP86 commitment, signed with internal key\n");

    // Just some addresses for outputs from our wallets. Not really important.
    let to_address =
        Address::from_str("bcrt1p5ukclgssqxk8mpk4830ujzmp5xw4zjzk5xhu9e43yf2hm5g7ecdq84tw3n")?
            .require_network(Network::Regtest)?;
    let change_address =
        Address::from_str("bcrt1ptkzxr3hmxru8xfpqje83mjwenj3lm484rejcuptch7w065ef3zns9wvwzh")?
            .require_network(Network::Regtest)?;
    let amount_to_send_in_sats = Amount::ONE_BTC;
    let change_amount = UTXO_1
        .amount_in_sats
        .checked_sub(amount_to_send_in_sats)
        .and_then(|x| x.checked_sub(ABSOLUTE_FEES_IN_SATS))
        .ok_or("Fees more than input amount!")?;

    let tx_hex_string = encode::serialize_hex(&generate_bip86_key_spend_tx(
        &secp,
        // The master extended private key from the descriptor in step 4
        Xpriv::from_str(BENEFACTOR_XPRIV_STR)?,
        // Set these fields with valid data for the UTXO from step 5 above
        UTXO_1,
        vec![
            TxOut { value: amount_to_send_in_sats, script_pubkey: to_address.script_pubkey() },
            TxOut { value: change_amount, script_pubkey: change_address.script_pubkey() },
        ],
    )?);
    println!(
        "\nYou should now be able to broadcast the following transaction: \n\n{}",
        tx_hex_string
    );

    println!("\nEND EXAMPLE 1\n");
    println!("----------------\n");




----------------

START EXAMPLE 1 - P2TR with a BIP86 commitment, signed with internal key


You should now be able to broadcast the following transaction: 

02000000000101122d59e6075aaceb96624bcce5f74fa14e2b7ebd0105ca46216ad0a5c4b9441f0000000000ffffffff0200e1f50500000000225120a72d8fa21001ac7d86d53c5fc90b61a19d514856a1afc2e6b122557dd11ece1a18140d8f000000002251205d8461c6fb30f8732420964f1dc9d99ca3fdd4f51e658e0578bf9cfd532988a7014100cf52f885085edeb64f971ba1a3be077a9d3e2bcc8e007d99396de6291e2ce58731079bf6477240fc69808c73003e745bc125b35d0e4a0fa9ff4e8af3fb87330100000000

END EXAMPLE 1

----------------


In [14]:
/// A wallet that allows creating and spending from an inheritance directly via the key path for purposes
/// of refreshing the inheritance timelock or changing other spending conditions.
struct BenefactorWallet {
    master_xpriv: Xpriv,
    beneficiary_xpub: Xpub,
    current_spend_info: Option<TaprootSpendInfo>,
    next_psbt: Option<Psbt>,
    secp: Secp256k1<secp256k1::All>,
    next: ChildNumber,
}

impl BenefactorWallet {
    fn new(
        master_xpriv: Xpriv,
        beneficiary_xpub: Xpub,
    ) -> Result<Self, Box<dyn std::error::Error>> {
        Ok(Self {
            master_xpriv,
            beneficiary_xpub,
            current_spend_info: None,
            next_psbt: None,
            secp: Secp256k1::new(),
            next: ChildNumber::from_normal_idx(0).expect("Zero is a valid child number"),
        })
    }

    fn time_lock_script(
        locktime: absolute::LockTime,
        beneficiary_key: XOnlyPublicKey,
    ) -> ScriptBuf {
        script::Builder::new()
            .push_int(locktime.to_consensus_u32() as i64)
            .push_opcode(OP_CLTV)
            .push_opcode(OP_DROP)
            .push_x_only_key(&beneficiary_key)
            .push_opcode(OP_CHECKSIG)
            .into_script()
    }

    fn create_inheritance_funding_tx(
        &mut self,
        lock_time: absolute::LockTime,
        input_utxo: P2trUtxo,
    ) -> Result<(Transaction, Psbt), Box<dyn std::error::Error>> {
        if let ChildNumber::Normal { index } = self.next {
            if index > 0 && self.current_spend_info.is_some() {
                return Err("Transaction already exists, use refresh_inheritance_timelock to refresh the timelock".into());
            }
        }
        // We use some other derivation path in this example for our inheritance protocol. The important thing is to ensure
        // that we use an unhardened path so we can make use of xpubs.
        let derivation_path = DerivationPath::from_str(&format!("m/101/1/0/0/{}", self.next))?;
        let internal_keypair =
            self.master_xpriv.derive_priv(&self.secp, &derivation_path)?.to_keypair(&self.secp);
        let beneficiary_key =
            self.beneficiary_xpub.derive_pub(&self.secp, &derivation_path)?.to_x_only_pub();

        // Build up the leaf script and combine with internal key into a taproot commitment
        let script = Self::time_lock_script(lock_time, beneficiary_key);
        let leaf_hash = script.tapscript_leaf_hash();

        let taproot_spend_info = TaprootBuilder::new()
            .add_leaf(0, script.clone())?
            .finalize(&self.secp, internal_keypair.x_only_public_key().0)
            .expect("Should be finalizable");
        self.current_spend_info = Some(taproot_spend_info.clone());
        let script_pubkey = ScriptBuf::new_p2tr(
            &self.secp,
            taproot_spend_info.internal_key(),
            taproot_spend_info.merkle_root(),
        );
        let value = input_utxo.amount_in_sats - ABSOLUTE_FEES_IN_SATS;

        // Spend a normal BIP86-like output as an input in our inheritance funding transaction
        let tx = generate_bip86_key_spend_tx(
            &self.secp,
            self.master_xpriv,
            input_utxo,
            vec![TxOut { script_pubkey: script_pubkey.clone(), value }],
        )?;

        // CREATOR + UPDATER
        let next_tx = Transaction {
            version: transaction::Version::TWO,
            lock_time,
            input: vec![TxIn {
                previous_output: OutPoint { txid: tx.txid(), vout: 0 },
                script_sig: ScriptBuf::new(),
                sequence: bitcoin::Sequence(0xFFFFFFFD), // enable locktime and opt-in RBF
                witness: Witness::default(),
            }],
            output: vec![],
        };
        let mut next_psbt = Psbt::from_unsigned_tx(next_tx)?;
        let mut origins = BTreeMap::new();
        origins.insert(
            beneficiary_key,
            (vec![leaf_hash], (self.beneficiary_xpub.fingerprint(), derivation_path.clone())),
        );
        origins.insert(
            internal_keypair.x_only_public_key().0,
            (vec![], (self.master_xpriv.fingerprint(&self.secp), derivation_path)),
        );
        let ty = PsbtSighashType::from_str("SIGHASH_ALL")?;
        let mut tap_scripts = BTreeMap::new();
        tap_scripts.insert(
            taproot_spend_info.control_block(&(script.clone(), LeafVersion::TapScript)).unwrap(),
            (script, LeafVersion::TapScript),
        );

        let input = Input {
            witness_utxo: { Some(TxOut { value, script_pubkey }) },
            tap_key_origins: origins,
            tap_merkle_root: taproot_spend_info.merkle_root(),
            sighash_type: Some(ty),
            tap_internal_key: Some(internal_keypair.x_only_public_key().0),
            tap_scripts,
            ..Default::default()
        };

        next_psbt.inputs = vec![input];
        self.next_psbt = Some(next_psbt.clone());

        self.next.increment()?;
        Ok((tx, next_psbt))
    }

    fn refresh_tx(
        &mut self,
        lock_time_delta: u32,
    ) -> Result<(Transaction, Psbt), Box<dyn std::error::Error>> {
        if let Some(ref spend_info) = self.current_spend_info.clone() {
            let mut psbt = self.next_psbt.clone().expect("Should have next_psbt");
            let input = &mut psbt.inputs[0];
            let input_value = input.witness_utxo.as_ref().unwrap().value;
            let output_value = input_value - ABSOLUTE_FEES_IN_SATS;

            // We use some other derivation path in this example for our inheritance protocol. The important thing is to ensure
            // that we use an unhardened path so we can make use of xpubs.
            let new_derivation_path =
                DerivationPath::from_str(&format!("m/101/1/0/0/{}", self.next))?;
            let new_internal_keypair = self
                .master_xpriv
                .derive_priv(&self.secp, &new_derivation_path)?
                .to_keypair(&self.secp);
            let beneficiary_key =
                self.beneficiary_xpub.derive_pub(&self.secp, &new_derivation_path)?.to_x_only_pub();

            // Build up the leaf script and combine with internal key into a taproot commitment
            let lock_time = absolute::LockTime::from_height(
                psbt.unsigned_tx.lock_time.to_consensus_u32() + lock_time_delta,
            )
            .unwrap();
            let script = Self::time_lock_script(lock_time, beneficiary_key);
            let leaf_hash = script.tapscript_leaf_hash();

            let taproot_spend_info = TaprootBuilder::new()
                .add_leaf(0, script.clone())?
                .finalize(&self.secp, new_internal_keypair.x_only_public_key().0)
                .expect("Should be finalizable");
            self.current_spend_info = Some(taproot_spend_info.clone());
            let prevout_script_pubkey = input.witness_utxo.as_ref().unwrap().script_pubkey.clone();
            let output_script_pubkey = ScriptBuf::new_p2tr(
                &self.secp,
                taproot_spend_info.internal_key(),
                taproot_spend_info.merkle_root(),
            );

            psbt.unsigned_tx.output =
                vec![TxOut { script_pubkey: output_script_pubkey.clone(), value: output_value }];
            psbt.outputs = vec![Output::default()];
            psbt.unsigned_tx.lock_time = absolute::LockTime::ZERO;

            let hash_ty = input
                .sighash_type
                .and_then(|psbt_sighash_type| psbt_sighash_type.taproot_hash_ty().ok())
                .unwrap_or(TapSighashType::All);
            let hash = SighashCache::new(&psbt.unsigned_tx).taproot_key_spend_signature_hash(
                0,
                &sighash::Prevouts::All(&[TxOut {
                    value: input_value,
                    script_pubkey: prevout_script_pubkey,
                }]),
                hash_ty,
            )?;

            {
                let (_, (_, derivation_path)) = input
                    .tap_key_origins
                    .get(&input.tap_internal_key.ok_or("Internal key missing in PSBT")?)
                    .ok_or("Missing taproot key origin")?;
                let secret_key =
                    self.master_xpriv.derive_priv(&self.secp, &derivation_path)?.to_priv().inner;
                sign_psbt_taproot(
                    &secret_key,
                    spend_info.internal_key(),
                    None,
                    input,
                    hash,
                    hash_ty,
                    &self.secp,
                );
            }

            // FINALIZER
            psbt.inputs.iter_mut().for_each(|input| {
                let mut script_witness: Witness = Witness::new();
                script_witness.push(input.tap_key_sig.unwrap().to_vec());
                input.final_script_witness = Some(script_witness);

                // Clear all the data fields as per the spec.
                input.partial_sigs = BTreeMap::new();
                input.sighash_type = None;
                input.redeem_script = None;
                input.witness_script = None;
                input.bip32_derivation = BTreeMap::new();
            });

            // EXTRACTOR
            let tx = psbt.extract_tx_unchecked_fee_rate();
            tx.verify(|_| {
                Some(TxOut { value: input_value, script_pubkey: output_script_pubkey.clone() })
            })
            .expect("failed to verify transaction");

            let next_tx = Transaction {
                version: transaction::Version::TWO,
                lock_time,
                input: vec![TxIn {
                    previous_output: OutPoint { txid: tx.txid(), vout: 0 },
                    script_sig: ScriptBuf::new(),
                    sequence: bitcoin::Sequence(0xFFFFFFFD), // enable locktime and opt-in RBF
                    witness: Witness::default(),
                }],
                output: vec![],
            };
            let mut next_psbt = Psbt::from_unsigned_tx(next_tx)?;
            let mut origins = BTreeMap::new();
            origins.insert(
                beneficiary_key,
                (vec![leaf_hash], (self.beneficiary_xpub.fingerprint(), new_derivation_path)),
            );
            let ty = PsbtSighashType::from_str("SIGHASH_ALL")?;
            let mut tap_scripts = BTreeMap::new();
            tap_scripts.insert(
                taproot_spend_info
                    .control_block(&(script.clone(), LeafVersion::TapScript))
                    .unwrap(),
                (script, LeafVersion::TapScript),
            );

            let input = Input {
                witness_utxo: {
                    let script_pubkey = output_script_pubkey;
                    let amount = output_value;

                    Some(TxOut { value: amount, script_pubkey })
                },
                tap_key_origins: origins,
                tap_merkle_root: taproot_spend_info.merkle_root(),
                sighash_type: Some(ty),
                tap_internal_key: Some(new_internal_keypair.x_only_public_key().0),
                tap_scripts,
                ..Default::default()
            };

            next_psbt.inputs = vec![input];
            self.next_psbt = Some(next_psbt.clone());

            self.next.increment()?;
            Ok((tx, next_psbt))
        } else {
            Err("No current_spend_info available. Create an inheritance tx first.".into())
        }
    }
}



In [15]:

/// A wallet that allows spending from an inheritance locked to a P2TR UTXO via a script path
/// after some expiry using CLTV.
struct BeneficiaryWallet {
    master_xpriv: Xpriv,
    secp: secp256k1::Secp256k1<secp256k1::All>,
}

impl BeneficiaryWallet {
    fn new(master_xpriv: Xpriv) -> Result<Self, Box<dyn std::error::Error>> {
        Ok(Self { master_xpriv, secp: Secp256k1::new() })
    }

    fn master_xpub(&self) -> Xpub { Xpub::from_priv(&self.secp, &self.master_xpriv) }

    fn spend_inheritance(
        &self,
        mut psbt: Psbt,
        lock_time: absolute::LockTime,
        to_address: Address,
    ) -> Result<Transaction, Box<dyn std::error::Error>> {
        let input_value = psbt.inputs[0].witness_utxo.as_ref().unwrap().value;
        let input_script_pubkey =
            psbt.inputs[0].witness_utxo.as_ref().unwrap().script_pubkey.clone();
        psbt.unsigned_tx.lock_time = lock_time;
        psbt.unsigned_tx.output = vec![TxOut {
            script_pubkey: to_address.script_pubkey(),
            value: input_value - ABSOLUTE_FEES_IN_SATS,
        }];
        psbt.outputs = vec![Output::default()];
        let unsigned_tx = psbt.unsigned_tx.clone();

        // SIGNER
        for (x_only_pubkey, (leaf_hashes, (_, derivation_path))) in
            &psbt.inputs[0].tap_key_origins.clone()
        {
            let secret_key =
                self.master_xpriv.derive_priv(&self.secp, &derivation_path)?.to_priv().inner;
            for lh in leaf_hashes {
                let hash_ty = TapSighashType::All;
                let hash = SighashCache::new(&unsigned_tx).taproot_script_spend_signature_hash(
                    0,
                    &sighash::Prevouts::All(&[TxOut {
                        value: input_value,
                        script_pubkey: input_script_pubkey.clone(),
                    }]),
                    *lh,
                    hash_ty,
                )?;
                sign_psbt_taproot(
                    &secret_key,
                    *x_only_pubkey,
                    Some(*lh),
                    &mut psbt.inputs[0],
                    hash,
                    hash_ty,
                    &self.secp,
                );
            }
        }

        // FINALIZER
        psbt.inputs.iter_mut().for_each(|input| {
            let mut script_witness: Witness = Witness::new();
            for (_, signature) in input.tap_script_sigs.iter() {
                script_witness.push(signature.to_vec());
            }
            for (control_block, (script, _)) in input.tap_scripts.iter() {
                script_witness.push(script.to_bytes());
                script_witness.push(control_block.serialize());
            }
            input.final_script_witness = Some(script_witness);

            // Clear all the data fields as per the spec.
            input.partial_sigs = BTreeMap::new();
            input.sighash_type = None;
            input.redeem_script = None;
            input.witness_script = None;
            input.bip32_derivation = BTreeMap::new();
            input.tap_script_sigs = BTreeMap::new();
            input.tap_scripts = BTreeMap::new();
            input.tap_key_sig = None;
        });

        // EXTRACTOR
        let tx = psbt.extract_tx_unchecked_fee_rate();
        tx.verify(|_| {
            Some(TxOut { value: input_value, script_pubkey: input_script_pubkey.clone() })
        })
        .expect("failed to verify transaction");

        Ok(tx)
    }
}



In [16]:
    println!("START EXAMPLE 2 - Script path spending of inheritance UTXO\n");

    {
        let beneficiary = BeneficiaryWallet::new(Xpriv::from_str(BENEFICIARY_XPRIV_STR)?)?;

        let mut benefactor = BenefactorWallet::new(
            Xpriv::from_str(BENEFACTOR_XPRIV_STR)?,
            beneficiary.master_xpub(),
        )?;
        let (tx, psbt) = benefactor.create_inheritance_funding_tx(
            absolute::LockTime::from_height(1000).unwrap(),
            UTXO_2,
        )?;
        let tx_hex = encode::serialize_hex(&tx);

        println!("Inheritance funding tx hex:\n\n{}", tx_hex);
        // You can now broadcast the transaction hex:
        // bt sendrawtransaction ...
        //
        // And mine a block to confirm the transaction:
        // bt generatetoaddress 1 $(bt-benefactor getnewaddress '' 'bech32m')

        let spending_tx = beneficiary.spend_inheritance(
            psbt,
            absolute::LockTime::from_height(1000).unwrap(),
            to_address,
        )?;
        let spending_tx_hex = encode::serialize_hex(&spending_tx);
        println!("\nInheritance spending tx hex:\n\n{}", spending_tx_hex);
        // If you try to broadcast now, the transaction will be rejected as it is timelocked.
        // First mine 900 blocks so we're sure we are over the 1000 block locktime:
        // bt generatetoaddress 900 $(bt-benefactor getnewaddress '' 'bech32m')
        // Then broadcast the transaction with `bt sendrawtransaction ...`
    }

    println!("\nEND EXAMPLE 2\n");
    println!("----------------\n");



START EXAMPLE 2 - Script path spending of inheritance UTXO

Inheritance funding tx hex:

02000000000101d8407b7093f79b42873d306ef7126640c01b9d4f907113e28ea85e5eb7ad0d4a0000000000ffffffff0118f5029500000000225120c8802d868f0600a2368fdb953ece239d2226e8164be9183fc9849b4bae24401e0141b3741930df7cf1e2aa825f1db04ee5a544664a0bee318f8fd935fc798d6ed5e50f7722cee7907c7c08974e94c989bd637f12adbc2aff7cc1da0fb6dd5c32d4910100000000

Inheritance spending tx hex:

02000000000101603d7710ef83998c0bdb37bd63301705f27ef910d9a6dd7e37377181151755f70000000000fdffffff0130f1029500000000225120a72d8fa21001ac7d86d53c5fc90b61a19d514856a1afc2e6b122557dd11ece1a034149d1f382ea798fd181f96174b4d0203c73e8acfb8d9f5280627d11a7568ce39844547ea1826c79f5570cab22f5683a1bc240b28c1f7fd23471855db6288dc9b6012702e803b175200d581db6ff5e3970ca0eaa17c74426e271ac5f2f8309a4018311470d9c669cc0ac21c17f3e515f2aa028beb01b9b56716efbc57d8a11c0cb4c158f68348a060e62228ee8030000

END EXAMPLE 2

----------------


In [17]:
   println!("START EXAMPLE 3 - Key path spending of inheritance UTXO\n");

    {
        let beneficiary = BeneficiaryWallet::new(Xpriv::from_str(BENEFICIARY_XPRIV_STR)?)?;

        let mut benefactor = BenefactorWallet::new(
            Xpriv::from_str(BENEFACTOR_XPRIV_STR)?,
            beneficiary.master_xpub(),
        )?;
        let (tx, _) = benefactor.create_inheritance_funding_tx(
            absolute::LockTime::from_height(2000).unwrap(),
            UTXO_3,
        )?;
        let tx_hex = encode::serialize_hex(&tx);

        println!("Inheritance funding tx hex:\n\n{}", tx_hex);
        // You can now broadcast the transaction hex:
        // bt sendrawtransaction ...
        //
        // And mine a block to confirm the transaction:
        // bt generatetoaddress 1 $(bt-benefactor getnewaddress '' 'bech32m')

        // At some point we may want to extend the locktime further into the future for the beneficiary.
        // We can do this by "refreshing" the inheritance transaction as the benefactor. This effectively
        // spends the inheritance transaction via the key path of the taproot output, and is not encumbered
        // by the timelock so we can spend it immediately. We set up a new output similar to the first with
        // a locktime that is 'locktime_delta' blocks greater.
        let (tx, _) = benefactor.refresh_tx(1000)?;
        let tx_hex = encode::serialize_hex(&tx);

        println!("\nRefreshed inheritance tx hex:\n\n{}\n", tx_hex);

        println!("\nEND EXAMPLE 3\n");
        println!("----------------\n");
    }





START EXAMPLE 3 - Key path spending of inheritance UTXO

Inheritance funding tx hex:

02000000000101af2af93fb1e8a577a074079c064da0b1a02ba688aadca17968796e8a325f02550100000000ffffffff0118f50295000000002251208c9163d0144e872af2495ff4e12c5325dc39792ba1ad909fe3c03bd9fef8048101411e263d148c084f320d8bb382d6ac215697a2ea2049d5a6ec794f105f9a4161d0fcac89c71e2920bbe266df9a3865c8d525f35b52820faf2b81bea48cc7d2632e0100000000

Refreshed inheritance tx hex:

020000000001015c73c8ec0784e869310969898d615c2923fbfd7e1a996be995759e8146f87d000000000000fdffffff0130f1029500000000225120fe6e6e0aa9e1c1368f423217b0f436b4a5f9a53d9b2323275d60c01fef1f6f8f01413689ee2688f3019f7cfe4d636faa140e35380f836e8ab9a69f1d77ccf9590c1692225b0c7887db2d3079dde32b9c504e19e23d94ab0b62a5e37fd6983be498510100000000


END EXAMPLE 3

----------------


()